<a href="https://colab.research.google.com/github/RiyaH2020/Age-Invariant-Face-Recognition/blob/main/Vgg_face_final_AIFR_FG_NET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import ZeroPadding2D,Convolution2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Dropout,Softmax,Flatten,Activation,BatchNormalization
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import tensorflow.keras.backend as K

# Define VGG_FACE_MODEL architecture
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

# Load VGG Face model weights
model.load_weights('drive/My Drive/vgg_face_weights.h5')

This code defines a convolutional neural network (CNN) based on the VGG-Face model, which is designed for face recognition. It consists of several convolutional, max-pooling, and dropout layers to extract facial features from input images. After processing through these layers, the model uses a softmax activation function to classify faces into 2622 possible identities. Pre-trained weights are loaded to enable the model to perform accurate face recognition without retraining. The architecture is built using TensorFlow/Keras and the weights are loaded from an external file.


In [ ]:
vgg_face=Model(inputs=model.layers[0].input,outputs=model.layers[-2].output)

This transformation creates a feature extraction model from the VGG-Face model. Instead of outputting face identity probabilities, it now outputs a 4096-dimensional feature vector (the output of the penultimate layer). These feature vectors can be used for tasks like facial similarity comparison or clustering instead of classification.


In [ ]:
pip install mtcnn

     |████████████████████████████████| 2.3MB 9.6MB/s 


In [ ]:
from mtcnn.mtcnn import MTCNN
from PIL import Image
from numpy import asarray

### Face Feature Extraction and Encoding with VGG-Face

This code extracts faces from images in the training dataset using **MTCNN** and then encodes them using a **pre-trained VGG-Face model**.

**Steps:**

1. **Face Extraction**  
   - `extract_face` function reads an image, detects the face using MTCNN, crops it, resizes it to `(224, 224)`, and converts it to a NumPy array.

2. **Dataset Processing**  
   - Iterates through each person's folder in the training dataset.  
   - Maps each person to a unique label in `person_rep`.  
   - For each image, the face is extracted, preprocessed for VGG-Face, and encoded into a feature vector.  
   - Encoded features are appended to `x_train`, and the corresponding label is appended to `y_train`.

3. **Output**  
   - `x_train`: List of VGG-Face feature vectors.  
   - `y_train`: Corresponding labels for each face.  

This prepares the dataset for training or evaluating face recognition models.


In [ ]:
def extract_face(filename,required_size=(224,224)):
  image=Image.open(filename)
  img=asarray(image)
  detector=MTCNN()
  results=detector.detect_faces(img)
  x1,y1,width,height=results[0]['box']
  x1,y1=abs(x1),abs(y1)
  x2,y2=x1+width,y1+height
  face=img[y1:y2,x1:x2]
  img=Image.fromarray(face)
  img=img.resize(required_size)
  face_array=asarray(img)
  print(filename)
  return face_array



import os
count=0
x_train=[]
y_train=[]
person_folders=os.listdir('drive/My Drive/Dataset/Train_detect/')
person_rep=dict()
for i,person in enumerate(person_folders):
  person_rep[i]=person
  image_names=os.listdir('drive/My Drive/Dataset/Train_detect/'+person+'/')
  for image_name in image_names:
    img=extract_face('drive/My Drive/Dataset/Train_detect/'+person+'/'+image_name)
    count+=1
    print(count)
    img=np.expand_dims(img,axis=0)
    img=preprocess_input(img)
    img_encode=vgg_face(img)
    x_train.append(np.squeeze(K.eval(img_encode)).tolist())
    y_train.append(i)

### Face Feature Extraction and Encoding for Test Dataset

This code extracts and encodes faces from the **test dataset** using **MTCNN** and the pre-trained **VGG-Face model**.

**Steps:**

1. **Face Extraction**  
   - The `extract_face` function detects faces in an image, crops them, resizes to `(224, 224)`, and converts to a NumPy array.

2. **Test Dataset Processing**  
   - Iterates through each person's folder in the test dataset.  
   - For each image, extracts the face, preprocesses it for VGG-Face, and encodes it into a feature vector.  
   - Encoded features are appended to `x_test`, and the corresponding label is appended to `y_test`.

3. **Output**  
   - `x_test`: List of VGG-Face feature vectors for the test set.  
   - `y_test`: Corresponding labels for each face.

This prepares the test dataset for evaluating the face recognition model.


In [ ]:
def extract_face(filename,required_size=(224,224)):
  image=Image.open(filename)
  img=asarray(image)
  detector=MTCNN()
  results=detector.detect_faces(img)
  x1,y1,width,height=results[0]['box']
  x1,y1=abs(x1),abs(y1)
  x2,y2=x1+width,y1+height
  face=img[y1:y2,x1:x2]
  img=Image.fromarray(face)
  img=img.resize(required_size)
  face_array=asarray(img)
  print(filename)
  return face_array



import os
count=0
x_test=[]
y_test=[]
person_folders=os.listdir('drive/My Drive/Dataset/Train_detect/')
for i,person in enumerate(person_folders):
  if os.path.exists('drive/My Drive/Dataset/Test_detect/'+person+'/'):
    image_names=os.listdir('drive/My Drive/Dataset/Test_detect/'+person+'/')
    for image_name in image_names:
      img=extract_face('drive/My Drive/Dataset/Test_detect/'+person+'/'+image_name)
      count+=1
      print(count)
      img=np.expand_dims(img,axis=0)
      img=preprocess_input(img)
      img_encode=vgg_face(img)
      x_test.append(np.squeeze(K.eval(img_encode)).tolist())
      y_test.append(i)

### VGG-Face Feature Extraction from Training Images (Without MTCNN)

This code encodes training images using the pre-trained **VGG-Face model** without explicit face detection.  

**Steps:**

1. **Dataset Loading**  
   - Iterates through each person's folder in the training dataset.  
   - Maps each person to a unique label in `person_rep`.  

2. **Image Processing**  
   - Loads each image and resizes it to `(224, 224)`.  
   - Converts the image to a NumPy array, expands dimensions, and applies **VGG-Face preprocessing**.  

3. **Feature Encoding**  
   - Encodes each image using the VGG-Face model to extract a feature vector.  
   - Appends the encoded feature to `x_train` and the corresponding label to `y_train`.  

**Output:**  
- `x_train`: List of VGG-Face feature vectors for the training set.  
- `y_train`: Corresponding labels for each image.  

This prepares the training dataset for training or evaluating a face recognition model.


In [ ]:
import os
x_train=[]
y_train=[]
person_folders=os.listdir('drive/My Drive/Dataset/Train_detect/')
person_rep=dict()
for i,person in enumerate(person_folders):
  person_rep[i]=person
  image_names=os.listdir('drive/My Drive/Dataset/Train_detect/'+person+'/')
  for image_name in image_names:
    print(image_name)
    img=load_img('drive/My Drive/Dataset/Train_detect/'+person+'/'+image_name,target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=preprocess_input(img)
    img_encode=vgg_face(img)
    x_train.append(np.squeeze(K.eval(img_encode)).tolist())
    y_train.append(i)



In [ ]:
print(img_encode)

tf.Tensor(
[[ 3.7940671   0.32182863  0.18288684 ... -3.5844185  -0.5676227
   1.2006351 ]], shape=(1, 2622), dtype=float32)


In [ ]:
person_rep

{0: '1',
 1: '2',
 2: '3',
 3: '4',
 4: '5',
 5: '6',
 6: '8',
 7: '9',
 8: '10',
 9: '11',
 10: '12',
 11: '13',
 12: '14',
 13: '15',
 14: '16',
 15: '17',
 16: '18',
 17: '19',
 18: '20',
 19: '21',
 20: '22',
 21: '23',
 22: '24',
 23: '25',
 24: '26',
 25: '27',
 26: '28',
 27: '29',
 28: '30',
 29: '31',
 30: '32',
 31: '33',
 32: '34',
 33: '35',
 34: '36',
 35: '37',
 36: '38',
 37: '39',
 38: '40',
 39: '41',
 40: '42',
 41: '43',
 42: '44',
 43: '45',
 44: '46',
 45: '47',
 46: '48',
 47: '49',
 48: '50',
 49: '51',
 50: '52',
 51: '53',
 52: '54',
 53: '55',
 54: '56',
 55: '57',
 56: '58',
 57: '59',
 58: '60',
 59: '61',
 60: '62',
 61: '63',
 62: '64',
 63: '65',
 64: '66',
 65: '67',
 66: '68',
 67: '69',
 68: '70',
 69: '71',
 70: '72',
 71: '73',
 72: '7',
 73: '74',
 74: '75',
 75: '76',
 76: '77',
 77: '78',
 78: '79',
 79: '80',
 80: '81',
 81: '82'}

In [ ]:
x_train=np.array(x_train)
y_train=np.array(y_train)


### VGG-Face Feature Extraction from Test Images (Without MTCNN)

This code encodes test images using the pre-trained **VGG-Face model** without explicit face detection.  

**Steps:**

1. **Dataset Loading**  
   - Iterates through each person's folder in the test dataset.  

2. **Image Processing**  
   - Loads each image and resizes it to `(224, 224)`.  
   - Converts the image to a NumPy array, expands dimensions, and applies **VGG-Face preprocessing**.  

3. **Feature Encoding**  
   - Encodes each image using the VGG-Face model to extract a feature vector.  
   - Appends the encoded feature to `x_test` and the corresponding label to `y_test`.  

**Output:**  
- `x_test`: List of VGG-Face feature vectors for the test set.  
- `y_test`: Corresponding labels for each image.  

This prepares the test dataset for evaluating the face recognition model.


In [ ]:
x_test=[]
y_test=[]
person_folders=os.listdir('drive/My Drive/Dataset/Train_detect/')
for i,person in enumerate(person_folders):
  image_names=os.listdir('drive/My Drive/Dataset/Test_detect/'+person+'/')
  for image_name in image_names:
    img=load_img('drive/My Drive/Dataset/Test_detect/'+person+'/'+image_name,target_size=(224,224))
    img=img_to_array(img)
    img=np.expand_dims(img,axis=0)
    img=preprocess_input(img)
    img_encode=vgg_face(img)
    x_test.append(np.squeeze(K.eval(img_encode)).tolist())
    y_test.append(i)

In [ ]:
x_test=np.array(x_test)
y_test=np.array(y_test)

### Fully Connected Classifier for Face Recognition

This code defines and compiles a **fully connected neural network** to classify face embeddings extracted from VGG-Face.

**Architecture:**

1. **Input Layer:**  
   - Takes VGG-Face feature vectors as input (`x_train.shape[1]`).  

2. **Hidden Layers:**  
   - **Dense Layer 1:** 1024 units, Glorot uniform initialization, followed by Batch Normalization, ReLU activation, and 30% Dropout.  
   - **Dense Layer 2:** 128 units, Glorot uniform initialization, followed by Batch Normalization, ReLU activation, and 20% Dropout.  

3. **Output Layer:**  
   - 82 units (number of classes), He uniform initialization, Softmax activation for multi-class classification.  

4. **Compilation:**  
   - Loss: `SparseCategoricalCrossentropy`  
   - Optimizer: `Nadam`  
   - Metrics: `Accuracy`  

This model takes precomputed face embeddings and learns to classify them into the correct person.


In [ ]:
classifier_model=Sequential()
classifier_model.add(Dense(units=1024,input_dim=x_train.shape[1],kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('relu'))
classifier_model.add(Dropout(0.3))
classifier_model.add(Dense(units=128,kernel_initializer='glorot_uniform'))
classifier_model.add(BatchNormalization())
classifier_model.add(Activation('relu'))
classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(units=82,kernel_initializer='he_uniform'))
classifier_model.add(Activation('softmax'))
classifier_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer='nadam',metrics=['accuracy'])

In [ ]:
history=classifier_model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100)

Epoch 1/100
23/23 [==============================] - 1s 54ms/step - loss: 3.9088 - accuracy: 0.1619 - val_loss: 4.5839 - val_accuracy: 0.2946
Epoch 2/100
23/23 [==============================] - 1s 43ms/step - loss: 2.2273 - accuracy: 0.5633 - val_loss: 2.3210 - val_accuracy: 0.5426
Epoch 3/100
23/23 [==============================] - 1s 43ms/step - loss: 1.5191 - accuracy: 0.7633 - val_loss: 1.9687 - val_accuracy: 0.5659
Epoch 4/100
23/23 [==============================] - 1s 43ms/step - loss: 1.1205 - accuracy: 0.8721 - val_loss: 1.7703 - val_accuracy: 0.6512
Epoch 5/100
23/23 [==============================] - 1s 44ms/step - loss: 0.8328 - accuracy: 0.9170 - val_loss: 1.6873 - val_accuracy: 0.6977
Epoch 6/100
23/23 [==============================] - 1s 44ms/step - loss: 0.6121 - accuracy: 0.9469 - val_loss: 1.5745 - val_accuracy: 0.7054
Epoch 7/100
23/23 [==============================] - 1s 44ms/step - loss: 0.4780 - accuracy: 0.9701 - val_loss: 1.4914 - val_accuracy: 0.7364
Epoch 

In [ ]:
_,train_acc=classifier_model.evaluate(x_train,y_train,verbose=0)
_,test_acc=classifier_model.evaluate(x_test,y_test,verbose=0)
print('Train:%.3f, Test:%.3f'%(train_acc*100,test_acc*100))

Train:100.000, Test:78.295


In [ ]:
pixels = load_img('drive/My Drive/Dataset/Test_detect/13/013A30.JPG',target_size=(224,224))
pixels = img_to_array(pixels)
samples = np.expand_dims(pixels, axis=0)
samples = preprocess_input(samples)
pixels_encode=vgg_face(samples)

In [ ]:
per=np.argmax(classifier_model.predict(pixels_encode),axis=-1)
print(per)

[11]


### K-Nearest Neighbors (KNN) Classifier for Face Recognition

This code trains a **KNN classifier** on the VGG-Face embeddings and evaluates its accuracy on the test set.

**Steps:**

1. **Model Training:**  
   - Initializes a `KNeighborsClassifier` with `n_neighbors=1`.  
   - Trains the model on the training embeddings `x_train` with labels `y_train`.  

2. **Evaluation:**  
   - Computes the accuracy on the test set `x_test` and `y_test`.  
   - Prints the resulting accuracy.  

This provides a simple baseline for face recognition using nearest-neighbor matching on VGG-Face features.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=1).fit(x_train,y_train)

accuracy=knn.score(x_test,y_test)
print(accuracy)

0.6666666666666666


In [ ]:
print(y_test)

[ 0  0  1  1  2  2  3  3  4  5  5  6  6  7  7  8  9  9 10 10 11 11 12 13
 14 14 15 15 16 16 17 18 18 19 19 20 21 21 22 23 23 24 24 26 27 27 28 29
 29 30 30 31 32 32 33 33 34 34 35 35 36 36 37 37 38 38 39 40 41 41 42 42
 43 43 44 45 45 46 46 47 47 48 49 49 50 51 51 52 52 53 54 55 56 57 58 59
 60 60 62 63 63 64 64 65 66 67 68 69 69 70 70 71 71 72 73 73 74 75 76 76
 77 77 78 78 79 80 80 81 81]


### Support Vector Machine (SVM) Classifier for Face Recognition

This code trains a **linear SVM classifier** on the VGG-Face embeddings and evaluates its performance.

**Steps:**

1. **Model Training:**  
   - Initializes an `SVC` with a linear kernel.  
   - Fits the model on the training embeddings `x_train` with labels `y_train`.  

2. **Prediction:**  
   - Predicts labels for both the training set (`yhat_train`) and the test set (`yhat_test`).  

3. **Evaluation:**  
   - Computes accuracy for training and test sets using `accuracy_score`.  
   - Prints the resulting training and test accuracies.  

This provides a robust approach for face recognition based on VGG-Face feature embeddings using SVM.


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
svmmodel=SVC(kernel='linear')
svmmodel.fit(x_train,y_train)
yhat_train=svmmodel.predict(x_train)
yhat_test=svmmodel.predict(x_test)
score_train=accuracy_score(y_train,yhat_train)
score_test=accuracy_score(y_test,yhat_test)
print('Accuracy: train=%.3f , test=%.3f' %(score_train*100, score_test*100))


Accuracy: train=100.000 , test=71.318
